In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.activations as activations
import tensorflow.keras.metrics as metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons.optimizers as optimizers
import tensorflow_addons.losses as losses
import numpy as np
import cv2


# Loads dataset.
# Targets are the segmantation images.
# Inputs are the original images.
def load_dataset(dataset_path):
    data = np.load(dataset_path, allow_pickle=True)
    return data['inputs'], data['targets']


# Defining the encoder's down-sampling blocks.
def encoder_block(inputs, n_filters, kernel_size, strides):
    encoder = layers.Conv2D(filters=n_filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(inputs)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation(activations.gelu)(encoder)
    encoder = layers.Conv2D(filters=n_filters, kernel_size=kernel_size, padding='same', use_bias=False)(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation(activations.gelu)(encoder)
    return encoder


# 定义解码器的上采样块。
def upscale_blocks(inputs):
    n_upscales = len(inputs)
    upscale_layers = []

    for i, inp in enumerate(inputs):
        p = n_upscales - i
        u = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2**p, padding='same')(inp)

        for i in range(2):
            u = layers.Conv2D(filters=64, kernel_size=3, padding='same', use_bias=False)(u)
            u = layers.BatchNormalization()(u)
            u = layers.Activation(activations.gelu)(u)
            u = layers.Dropout(rate=0.4)(u)

        upscale_layers.append(u)
    return upscale_layers


# 定义解码器的整个模块。

def decoder_block(layers_to_upscale, inputs):
    upscaled_layers = upscale_blocks(layers_to_upscale)

    decoder_blocks = []

    for i, inp in enumerate(inputs):
        d = layers.Conv2D(filters=64, kernel_size=3, strides=2**i, padding='same', use_bias=False)(inp)
        d = layers.BatchNormalization()(d)
        d = layers.Activation(activations.gelu)(d)
        d = layers.Conv2D(filters=64, kernel_size=3, padding='same', use_bias=False)(d)
        d = layers.BatchNormalization()(d)
        d = layers.Activation(activations.gelu)(d)

        decoder_blocks.append(d)

    decoder = layers.concatenate(upscaled_layers + decoder_blocks)
    decoder = layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', use_bias=False)(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation(activations.gelu)(decoder)
    decoder = layers.Dropout(rate=0.4)(decoder)

    return decoder


def get_model(input_dim):
    inputs = layers.Input(input_dim)

    noisy_inputs = layers.GaussianNoise(stddev=0.2)(inputs)

    e1 = encoder_block(noisy_inputs, n_filters=32, kernel_size=3, strides=1)
    e2 = encoder_block(e1, n_filters=64, kernel_size=3, strides=2)
    e3 = encoder_block(e2, n_filters=128, kernel_size=3, strides=2)
    e4 = encoder_block(e3, n_filters=256, kernel_size=3, strides=2)
    e5 = encoder_block(e4, n_filters=512, kernel_size=3, strides=2)

    d4 = decoder_block(layers_to_upscale=[e5], inputs=[e4, e3, e2, e1])
    d3 = decoder_block(layers_to_upscale=[e5, d4], inputs=[e3, e2, e1])
    d2 = decoder_block(layers_to_upscale=[e5, d4, d3], inputs=[e2, e1])
    d1 = decoder_block(layers_to_upscale=[e5, d4, d3, d2], inputs=[e1])

    output = layers.Conv2D(filters=3, kernel_size=1, padding='same', activation='tanh')(d1)

    model = models.Model(inputs, output)
    return model


model = get_model((256, 256, 3))
model.summary()


# Loading the dataset.
# x_train: Segmentation images.
# y_train: Original images.
DATASET_PATH = 'D:\\Datasets\\City-Segmentation\\Cityscapes\\dataset.npz'
x_train, y_train = load_dataset(DATASET_PATH)

# Normalizing data.
x_train = (x_train - 127.5) / 127.5
y_train = (y_train - 127.5) / 127.5

# Building & Compiling the model.
image_size = x_train[0].shape
unet3_plus = get_model(image_size)

unet3_plus.compile(
    optimizer=optimizers.Yogi(learning_rate=0.00025),
    loss=losses.sigmoid_focal_crossentropy,
    metrics=[metrics.MeanIoU(num_classes=30)]
)

# Training the model
epochs = 200
batch_size = 4

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=x_train.shape[0])
inputs = dataset.batch(batch_size=batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
batches_per_epoch = x_train.shape[0] // batch_size

for epoch in range(epochs):
    print('\nTraining on epoch', epoch + 1)

    loss = 0
    for i, (x_batch, y_batch) in enumerate(inputs):
        loss = unet3_plus.train_on_batch(x_batch, y_batch)

        print('\rCurrent batch: {}/{} , loss = {}'.format(
            i+1,
            batches_per_epoch,
            loss, end='')
        )

In [ ]:
  # def add_prediction_op(self):
  #   logging.info("Model: depths {depths}, filters {filters}, "
  #          "filter size {kernel_size[0]}x{kernel_size[1]}, "
  #          "pool size: {pool_size[0]}x{pool_size[1]}, "
  #          "dilation rate: {dilation_rate[0]}x{dilation_rate[1]}".format(
  #           depths=self.depths,
  #           filters=self.filters_root,
  #           kernel_size=self.kernel_size,
  #           dilation_rate=self.dilation_rate,
  #           pool_size=self.pool_size))


  #   # 如果 weight_decay 大于 0，设置 L2 正则化以防止过拟合，否则不使用正则化
  #   if self.weight_decay > 0:
  #     weight_decay = tf.constant(self.weight_decay, dtype=tf.float32, name="weight_constant")
  #     self.regularizer = tf.keras.regularizers.l2(l=0.5 * (weight_decay))
  #   else:
  #     self.regularizer = None

  #   #权重初始化器，1.0不缩放
  #   self.initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")



  #   # 输入层第一部分（3*3001-->8*3001）
  #   convs = [None] * self.depths # 存储每个深度的输出

  #   with tf.compat.v1.variable_scope("Input"):
  #     net = self.X

  #     #对输入数据应用卷积操作，滤波器数量为 filters_root，卷积核大小为 kernel_size，padding='same' 保持输出的空间尺寸与输入相同。
  #     net = tf.compat.v1.layers.conv2d(net,
  #                  filters=self.filters_root,
  #                  kernel_size=self.kernel_size,
  #                  activation=None,
  #                  padding='same',
  #                  dilation_rate=self.dilation_rate,
  #                  kernel_initializer=self.initializer,
  #                  kernel_regularizer=self.regularizer,
  #                  name="input_conv")
      
  #     #对卷积后的特征图进行批量归一化，以稳定训练过程，加快收敛。
  #     net = tf.compat.v1.layers.batch_normalization(net,
  #                       training=self.is_training,
  #                       name="input_bn")
  #     #应用 ReLU 激活函数，将非线性引入网络，增加模型表达能力。
  #     net = tf.nn.relu(net,
  #              name="input_relu")
  #     # net = tf.nn.dropout(net, self.keep_prob)
      
  #     #对特征图应用 Dropout，以防止过拟合，drop_rate 控制丢弃的比例。
  #     net = tf.compat.v1.layers.dropout(net,
  #                 rate=self.drop_rate,
  #                 training=self.is_training,
  #                 name="input_dropout")



  #   #上面为输入层，现在开始下采样层（包含了输入层第二部分(8*3001-->8*3001)）
  #   for depth in range(0, self.depths):
  #     with tf.compat.v1.variable_scope("DownConv_%d" % depth):
        
  #       filters = int(2**(depth) * self.filters_root)
        
  #       net = tf.compat.v1.layers.conv2d(net,
  #                    filters=filters,
  #                    kernel_size=self.kernel_size,
  #                    activation=None,
  #                    use_bias=False,
  #                    padding='same',
  #                    dilation_rate=self.dilation_rate,
  #                    kernel_initializer=self.initializer,
  #                    kernel_regularizer=self.regularizer,
  #                    name="down_conv1_{}".format(depth + 1))
  #       net = tf.compat.v1.layers.batch_normalization(net,
  #                         training=self.is_training,
  #                         name="down_bn1_{}".format(depth + 1))
  #       net = tf.nn.relu(net,
  #                name="down_relu1_{}".format(depth+1))
  #       net = tf.compat.v1.layers.dropout(net,
  #                   rate=self.drop_rate,
  #                   training=self.is_training,
  #                   name="down_dropout1_{}".format(depth + 1))

  #       convs[depth] = net 
  #       #每层网络结构convs储存
  #       #convs[0]是第二个8*3001,
  #       #convs[1]是11*751
  #       #convs[2]is 16*188
  #       #convs[3]is 22*47
  #       #convs[4]is 32*12
  #       #此刻convs储存了所有的需要skip connection的块


  #       #下层前四层0，1，2，3
  #       if depth < self.depths - 1:
  #         net = tf.compat.v1.layers.conv2d(net,
  #                      filters=filters,
  #                      kernel_size=self.kernel_size,
  #                      strides=self.pool_size, #Convolution+Stride+Rule
  #                      activation=None,
  #                      use_bias=False,
  #                      padding='same',
  #                      dilation_rate=self.dilation_rate,
  #                      kernel_initializer=self.initializer,
  #                      kernel_regularizer=self.regularizer,
  #                      name="down_conv3_{}".format(depth + 1))
  #         net = tf.compat.v1.layers.batch_normalization(net,
  #                           training=self.is_training,
  #                           name="down_bn3_{}".format(depth + 1))
  #         net = tf.nn.relu(net,
  #                  name="down_relu3_{}".format(depth+1))
  #         net = tf.compat.v1.layers.dropout(net,
  #                   rate=self.drop_rate,
  #                   training=self.is_training,
  #                   name="down_dropout3_{}".format(depth + 1))




  #   # 上层（4，3，2，1层）（3，2，1，0）
  #   for depth in range(self.depths - 2, -1, -1):
  #     with tf.compat.v1.variable_scope("UpConv_%d" % depth):
  #       filters = int(2**(depth) * self.filters_root) #64 32 16 8
  #       #卷积核随着深度增加而变化
  #       net = tf.compat.v1.layers.conv2d_transpose(net,
  #                        filters=filters,
  #                        kernel_size=self.kernel_size,
  #                        strides=self.pool_size,
  #                        activation=None,
  #                        use_bias=False,
  #                        padding="same",
  #                        kernel_initializer=self.initializer,
  #                        kernel_regularizer=self.regularizer,
  #                        name="up_conv0_{}".format(depth+1))
  #       net = tf.compat.v1.layers.batch_normalization(net,
  #                         training=self.is_training,
  #                         name="up_bn0_{}".format(depth + 1))
  #       net = tf.nn.relu(net,
  #                name="up_relu0_{}".format(depth+1))
  #       net = tf.compat.v1.layers.dropout(net,
  #                   rate=self.drop_rate,
  #                   training=self.is_training,
  #                   name="up_dropout0_{}".format(depth + 1))

        
  #       #跳跃连接
  #       #net>convs[depth]--size
  #       net = crop_and_concat(convs[depth], net)
  #       #net = crop_only(convs[depth], net)

  #       net = tf.compat.v1.layers.conv2d(net,
  #                    filters=filters,
  #                    kernel_size=self.kernel_size,
  #                    activation=None,
  #                    use_bias=False,
  #                    padding='same',
  #                    dilation_rate=self.dilation_rate,
  #                    kernel_initializer=self.initializer,
  #                    kernel_regularizer=self.regularizer,
  #                    name="up_conv1_{}".format(depth + 1))
  #       net = tf.compat.v1.layers.batch_normalization(net,
  #                         training=self.is_training,
  #                         name="up_bn1_{}".format(depth + 1))
  #       net = tf.nn.relu(net,
  #                name="up_relu1_{}".format(depth + 1))
  #       net = tf.compat.v1.layers.dropout(net,
  #                   rate=self.drop_rate,
  #                   training=self.is_training,
  #                   name="up_dropout1_{}".format(depth + 1))


  #   # Output Map
  #   with tf.compat.v1.variable_scope("Output"):
  #     net = tf.compat.v1.layers.conv2d(net,
  #                  filters=self.n_class,
  #                  kernel_size=(1,1),
  #                  activation=None,
  #                  padding='same',
  #                  #dilation_rate=self.dilation_rate,
  #                  kernel_initializer=self.initializer,
  #                  kernel_regularizer=self.regularizer,
  #                  name="output_conv")
  #     # net = tf.nn.relu(net,
  #     #                     name="output_relu")
  #     # net = tf.compat.v1.layers.dropout(net,
  #     #                         rate=self.drop_rate,
  #     #                         training=self.is_training,
  #     #                         name="output_dropout")
  #     # net = tf.compat.v1.layers.batch_normalization(net,
  #     #                                    training=self.is_training,
  #     #                                    name="output_bn")
  #     output = net



  #   # 保存编码器部分的最终表示，可以用于其他任务或分析
  #   with tf.compat.v1.variable_scope("representation"):
  #     self.representation = convs[-1]

  #   # logits 是最终的网络输出，未经过激活函数
  #   with tf.compat.v1.variable_scope("logits"):
  #     self.logits = output
  #     # 记录 logits 的直方图摘要，以便在 TensorBoard 中可视化
  #     tmp = tf.compat.v1.summary.histogram("logits", self.logits)
  #     self.summary_train.append(tmp)

  #   # 计算最终预测结果，使用 softmax 将 logits 转换为概率分布
  #   with tf.compat.v1.variable_scope("preds"):
  #     self.preds = tf.nn.softmax(output)
  #     # 记录预测结果的直方图摘要，以便在 TensorBoard 中可视化
  #     tmp = tf.compat.v1.summary.histogram("preds", self.preds)
  #     self.summary_train.append(tmp)


In [ ]:
  # def add_prediction_op(self):
  #   logging.info("Model: depths {depths}, filters {filters}, "
  #          "filter size {kernel_size[0]}x{kernel_size[1]}, "
  #          "pool size: {pool_size[0]}x{pool_size[1]}, "
  #          "dilation rate: {dilation_rate[0]}x{dilation_rate[1]}".format(
  #           depths=self.depths,
  #           filters=self.filters_root,
  #           kernel_size=self.kernel_size,
  #           dilation_rate=self.dilation_rate,
  #           pool_size=self.pool_size))


  #   # 如果 weight_decay 大于 0，设置 L2 正则化以防止过拟合，否则不使用正则化
  #   # 设置 L2 正则化
  #   self.regularizer = tf.keras.regularizers.l2(0.5 * self.weight_decay) if self.weight_decay > 0 else None


  #   # 初始化器
  #   self.initializer = tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")



  #   # 输入层第一部分（3*3001-->8*3001）
  #   convs = [None] * self.depths # 存储每个深度的输出

  #   with tf.name_scope("Input"):
  #     net = self.X  # 如果 self.X 是输入张量，直接赋值给 net

  #     net = tf.keras.layers.Conv2D(
  #                                  filters=self.filters_root, kernel_size=self.kernel_size, padding='same', dilation_rate=self.dilation_rate, kernel_initializer=self.initializer,kernel_regularizer=self.regularizer,name="input_conv")(net)
  #     net = tf.keras.layers.BatchNormalization( name="input_bn")(net, training=self.is_training)
  #     net = tf.keras.layers.ReLU(
  #                                name="input_relu")(net)
  #     net = tf.keras.layers.Dropout(rate=self.drop_rate)(net, training=self.is_training)






  #   # 下采样部分
  #   # 第一层
  #   with tf.name_scope("DownConv_0"):
  #       filters = int(2**0 * self.filters_root)  # 8
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_0_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_0_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_0_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_0_1")(net, training=self.is_training)

  #       convs[0] = net  # 存储跳跃连接块

  #       # 下采样卷积
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   strides=self.pool_size,  # Convolution+Stride
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_0_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_0_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_0_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_0_2")(net, training=self.is_training)

  #   # 第二层
  #   with tf.name_scope("DownConv_1"):
  #       filters = int(2**1 * self.filters_root)  # 16
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_1_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_1_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_1_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_1_1")(net, training=self.is_training)
  #       convs[1] = net  # 存储跳跃连接块

  #       # 下采样卷积
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   strides=self.pool_size,  # Convolution+Stride
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_1_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_1_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_1_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_1_2")(net, training=self.is_training)

  #   # 第三层
  #   with tf.name_scope("DownConv_2"):
  #       filters = int(2**2 * self.filters_root)  # 32
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_2_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_2_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_2_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_2_1")(net, training=self.is_training)
  #       convs[2] = net  # 存储跳跃连接块

  #       # 下采样卷积
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   strides=self.pool_size,  # Convolution+Stride
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_2_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_2_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_2_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_2_2")(net, training=self.is_training)

  #   # 第四层
  #   with tf.name_scope("DownConv_3"):
  #       filters = int(2**3 * self.filters_root)  # 64
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_3_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_3_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_3_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_3_1")(net, training=self.is_training)
  #       convs[3] = net  # 存储跳跃连接块

  #       # 下采样卷积
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   strides=self.pool_size,  # Convolution+Stride
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_3_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_3_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_3_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_3_2")(net, training=self.is_training)

  #   # 第五层
  #   with tf.name_scope("DownConv_4"):
  #       filters = int(2**4 * self.filters_root)  # 128
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   use_bias=False, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="down_conv_4_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="down_bn_4_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="down_relu_4_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="down_dropout_4_1")(net, training=self.is_training)
  #       convs[4] = net  # 存储跳跃连接块





  #     ## 上采样和跳跃连接部分
  #   # 第四层（从上至下，编号为 3）
  #   with tf.name_scope("UpConv_3"):
  #       filters = int(2**3 * self.filters_root)  # 64
  #       net = tf.keras.layers.Conv2DTranspose(filters=filters, 
  #                                             kernel_size=self.kernel_size, 
  #                                             strides=self.pool_size, 
  #                                             padding="same", 
  #                                             use_bias=False, 
  #                                             kernel_initializer=self.initializer, 
  #                                             kernel_regularizer=self.regularizer, 
  #                                             name="up_conv0_3")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn0_3")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu0_3")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout0_3")(net, training=self.is_training)
  #       #convs--8 16 32 64 128
  #       # 跳跃连接
  #       net = crop_and_concat(convs[3], net)

  #       # 卷积操作
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   use_bias=False, 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="up_conv1_3")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn1_3")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu1_3")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout1_3")(net, training=self.is_training)

  #   # 第三层
  #   with tf.name_scope("UpConv_2"):
  #       filters = int(2**2 * self.filters_root)  # 32
  #       net = tf.keras.layers.Conv2DTranspose(filters=filters, 
  #                                             kernel_size=self.kernel_size, 
  #                                             strides=self.pool_size, 
  #                                             padding="same", 
  #                                             use_bias=False, 
  #                                             kernel_initializer=self.initializer, 
  #                                             kernel_regularizer=self.regularizer, 
  #                                             name="up_conv0_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn0_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu0_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout0_2")(net, training=self.is_training)

  #       # 跳跃连接
  #       net = crop_and_concat(convs[2], net)

  #       # 卷积操作
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   use_bias=False, 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="up_conv1_2")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn1_2")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu1_2")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout1_2")(net, training=self.is_training)

  #   # 第二层
  #   with tf.name_scope("UpConv_1"):
  #       filters = int(2**1 * self.filters_root)  # 16
  #       net = tf.keras.layers.Conv2DTranspose(filters=filters, 
  #                                             kernel_size=self.kernel_size, 
  #                                             strides=self.pool_size, 
  #                                             padding="same", 
  #                                             use_bias=False, 
  #                                             kernel_initializer=self.initializer, 
  #                                             kernel_regularizer=self.regularizer, 
  #                                             name="up_conv0_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn0_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu0_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout0_1")(net, training=self.is_training)

  #       # 跳跃连接
  #       net = crop_and_concat(convs[1], net)

  #       # 卷积操作
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   use_bias=False, 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="up_conv1_1")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn1_1")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu1_1")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout1_1")(net, training=self.is_training)

  #   # 第一层
  #   with tf.name_scope("UpConv_0"):
  #       filters = int(2**0 * self.filters_root)  # 8
  #       net = tf.keras.layers.Conv2DTranspose(filters=filters, 
  #                                             kernel_size=self.kernel_size, 
  #                                             strides=self.pool_size, 
  #                                             padding="same", 
  #                                             use_bias=False, 
  #                                             kernel_initializer=self.initializer, 
  #                                             kernel_regularizer=self.regularizer, 
  #                                             name="up_conv0_0")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn0_0")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu0_0")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout0_0")(net, training=self.is_training)

  #       # 跳跃连接
  #       net = crop_and_concat(convs[0], net)

  #       # 卷积操作
  #       net = tf.keras.layers.Conv2D(filters=filters, 
  #                                   kernel_size=self.kernel_size, 
  #                                   padding='same', 
  #                                   use_bias=False, 
  #                                   dilation_rate=self.dilation_rate, 
  #                                   kernel_initializer=self.initializer, 
  #                                   kernel_regularizer=self.regularizer, 
  #                                   name="up_conv1_0")(net)
  #       net = tf.keras.layers.BatchNormalization(name="up_bn1_0")(net, training=self.is_training)
  #       net = tf.keras.layers.ReLU(name="up_relu1_0")(net)
  #       net = tf.keras.layers.Dropout(rate=self.drop_rate, name="up_dropout1_0")(net, training=self.is_training)


  #   #全连接








  #   # Output Map
  #   # 输出层
  #   with tf.name_scope("Output"):
  #       net = tf.keras.layers.Conv2D(filters=self.n_class,
  #                                   kernel_size=(1, 1),
  #                                   padding='same',
  #                                   activation=None,
  #                                   kernel_initializer=self.initializer,
  #                                   kernel_regularizer=self.regularizer,
  #                                   name="output_conv")(net)


  #   output = net



  #       # 保存编码器部分的最终表示，可以用于其他任务或分析
  #   with tf.name_scope("representation"):
  #       self.representation = convs[-1]

  #   # logits 是最终的网络输出，未经过激活函数
  #   with tf.name_scope("logits"):
  #       self.logits = output  # 假设 'net' 是模型最终输出的张量（output）

  #       # 记录 logits 的直方图摘要，以便在 TensorBoard 中可视化
  #       tf.summary.histogram("logits", self.logits, step=self.global_step)

  #   # 计算最终预测结果，使用 softmax 将 logits 转换为概率分布
  #   with tf.name_scope("preds"):
  #       self.preds = tf.nn.softmax(self.logits)

  #       # 记录预测结果的直方图摘要，以便在 TensorBoard 中可视化
  #       tf.summary.histogram("preds", self.preds, step=self.global_step)

In [4]:
"""
UNet3+ base model
"""
import tensorflow as tf
import tensorflow.keras as k

def conv_block(x, kernels, kernel_size=(3, 3), strides=(1, 1), padding='same',
               is_bn=True, is_relu=True, n=2):
    """ Custom function for conv2d:
        Apply  3*3 convolutions with BN and relu.
    """
    for i in range(1, n + 1):
        x = k.layers.Conv2D(filters=kernels, kernel_size=kernel_size,
                            padding=padding, strides=strides,
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                            kernel_initializer=k.initializers.he_normal(seed=5))(x)
        if is_bn:
            x = k.layers.BatchNormalization()(x)
        if is_relu:
            x = k.activations.relu(x)

    return x



def unet3plus(input_shape, output_channels):
    """ UNet3+ base model """
    filters = [64, 128, 256, 512, 1024]

    input_layer = k.layers.Input(
        shape=input_shape,
        name="input_layer"
    )  # 320*320*3

    """ Encoder"""
    # block 1
    e1 = conv_block(input_layer, filters[0])  # 320*320*64

    # block 2
    e2 = k.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 160*160*64
    e2 = conv_block(e2, filters[1])  # 160*160*128

    # block 3
    e3 = k.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 80*80*128
    e3 = conv_block(e3, filters[2])  # 80*80*256

    # block 4
    e4 = k.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 40*40*256
    e4 = conv_block(e4, filters[3])  # 40*40*512

    # block 5
    # bottleneck layer
    e5 = k.layers.MaxPool2D(pool_size=(2, 2))(e4)  # 20*20*512
    e5 = conv_block(e5, filters[4])  # 20*20*1024   

    """ Decoder """
    cat_channels = filters[0]
    cat_blocks = len(filters)
    upsample_channels = cat_blocks * cat_channels

    """ d4 """
    e1_d4 = k.layers.MaxPool2D(pool_size=(8, 8))(e1)  # 320*320*64  --> 40*40*64
    e1_d4 = conv_block(e1_d4, cat_channels, n=1)  # 320*320*64  --> 40*40*64

    e2_d4 = k.layers.MaxPool2D(pool_size=(4, 4))(e2)  # 160*160*128 --> 40*40*128
    e2_d4 = conv_block(e2_d4, cat_channels, n=1)  # 160*160*128 --> 40*40*64

    e3_d4 = k.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 80*80*256  --> 40*40*256
    e3_d4 = conv_block(e3_d4, cat_channels, n=1)  # 80*80*256  --> 40*40*64

    e4_d4 = conv_block(e4, cat_channels, n=1)  # 40*40*512  --> 40*40*64

    e5_d4 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(e5)  # 80*80*256  --> 40*40*256
    e5_d4 = conv_block(e5_d4, cat_channels, n=1)  # 20*20*1024  --> 20*20*64

    d4 = k.layers.concatenate([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = conv_block(d4, upsample_channels, n=1)  # 40*40*320  --> 40*40*320

    """ d3 """
    e1_d3 = k.layers.MaxPool2D(pool_size=(4, 4))(e1)  # 320*320*64 --> 80*80*64
    e1_d3 = conv_block(e1_d3, cat_channels, n=1)  # 80*80*64 --> 80*80*64

    e2_d3 = k.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 160*160*256 --> 80*80*256
    e2_d3 = conv_block(e2_d3, cat_channels, n=1)  # 80*80*256 --> 80*80*64

    e3_d3 = conv_block(e3, cat_channels, n=1)  # 80*80*512 --> 80*80*64

    e4_d3 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d4)  # 40*40*320 --> 80*80*320
    e4_d3 = conv_block(e4_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    e5_d3 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(e5)  # 20*20*320 --> 80*80*320
    e5_d3 = conv_block(e5_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    d3 = k.layers.concatenate([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = conv_block(d3, upsample_channels, n=1)  # 80*80*320 --> 80*80*320

    """ d2 """
    e1_d2 = k.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 320*320*64 --> 160*160*64
    e1_d2 = conv_block(e1_d2, cat_channels, n=1)  # 160*160*64 --> 160*160*64

    e2_d2 = conv_block(e2, cat_channels, n=1)  # 160*160*256 --> 160*160*64

    d3_d2 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d3)  # 80*80*320 --> 160*160*320
    d3_d2 = conv_block(d3_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d4_d2 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d4)  # 40*40*320 --> 160*160*320
    d4_d2 = conv_block(d4_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    e5_d2 = k.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(e5)  # 20*20*320 --> 160*160*320
    e5_d2 = conv_block(e5_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d2 = k.layers.concatenate([e1_d2, e2_d2, d3_d2, d4_d2, e5_d2])
    d2 = conv_block(d2, upsample_channels, n=1)  # 160*160*320 --> 160*160*320

    """ d1 """
    e1_d1 = conv_block(e1, cat_channels, n=1)  # 320*320*64 --> 320*320*64

    d2_d1 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d2)  # 160*160*320 --> 320*320*320
    d2_d1 = conv_block(d2_d1, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d3_d1 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d3)  # 80*80*320 --> 320*320*320
    d3_d1 = conv_block(d3_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d4_d1 = k.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(d4)  # 40*40*320 --> 320*320*320
    d4_d1 = conv_block(d4_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    e5_d1 = k.layers.UpSampling2D(size=(16, 16), interpolation='bilinear')(e5)  # 20*20*320 --> 320*320*320
    e5_d1 = conv_block(e5_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d1 = k.layers.concatenate([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1, ])
    d1 = conv_block(d1, upsample_channels, n=1)  # 320*320*320 --> 320*320*320

    # last layer does not have batchnorm and relu
    d = conv_block(d1, output_channels, n=1, is_bn=False, is_relu=False)

    output = k.activations.softmax(d)

    return tf.keras.Model(inputs=input_layer, outputs=[output], name='UNet_3Plus')


def tiny_unet3plus(input_shape, output_channels, training):
    """ Sample model only for testing during development """
    filters = [64, 128, 256, 512, 1024]

    input_layer = k.layers.Input(shape=input_shape, name="input_layer")  # 320*320*3

    """ Encoder"""
    # block 1
    e1 = conv_block(input_layer, filters[0] // 2)  # 320*320*64
    e1 = conv_block(e1, filters[0] // 2)  # 320*320*64

    # last layer does not have batch norm and relu
    d = conv_block(e1, output_channels, n=1, is_bn=False, is_relu=False)
    output = k.activations.softmax(d, )

    if training:
        e2 = conv_block(e1, filters[0] // 2)  # 320*320*64
        d2 = conv_block(e2, output_channels, n=1, is_bn=False, is_relu=False)
        output2 = k.activations.softmax(d2)
        #output2=k.activations.sigmoid(d2)
        #output2=k.activations.tanh(d2)
        return tf.keras.Model(inputs=input_layer, outputs=[output, output2], name='UNet3Plus')
    else:
        return tf.keras.Model(inputs=input_layer, outputs=[output], name='UNet3Plus')


if __name__ == "__main__":
    """## Model Compilation"""
    INPUT_SHAPE = [320, 320, 1]
    OUTPUT_CHANNELS = 1

    unet_3P = unet3plus(INPUT_SHAPE, OUTPUT_CHANNELS)
    unet_3P.summary()

    # tf.keras.utils.plot_model(unet_3P, show_layer_names=True, show_shapes=True)

    # unet_3P.save("unet_3P.hdf5")

Model: "UNet_3Plus"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 320, 320, 1)]        0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 320, 320, 64)         640       ['input_layer[0][0]']         
                                                                                                  
 batch_normalization_2 (Bat  (None, 320, 320, 64)         256       ['conv2d_2[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 tf.nn.relu_2 (TFOpLambda)   (None, 320, 320, 64)         0         ['batch_normalization

In [1]:
"""
UNet3+ with Deep Supervision
"""
import tensorflow as tf
import tensorflow.keras as k

def conv_block(x, kernels, kernel_size=(3, 3), strides=(1, 1), padding='same',
               is_bn=True, is_relu=True, n=2):
    """ Custom function for conv2d:
        Apply  3*3 convolutions with BN and relu.
    """
    for i in range(1, n + 1):
        x = k.layers.Conv2D(filters=kernels, kernel_size=kernel_size,
                            padding=padding, strides=strides,
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                            kernel_initializer=k.initializers.he_normal(seed=5))(x)
        if is_bn:
            x = k.layers.BatchNormalization()(x)
        if is_relu:
            x = k.activations.relu(x)

    return x


def unet3plus_deepsup(input_shape, output_channels, training=False):
    """ UNet_3Plus with Deep Supervision """
    filters = [64, 128, 256, 512, 1024]

    input_layer = k.layers.Input(shape=input_shape, name="input_layer")  # 320*320*3

    """ Encoder"""
    # block 1
    e1 = conv_block(input_layer, filters[0])  # 320*320*64

    # block 2
    e2 = k.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 160*160*64
    e2 = conv_block(e2, filters[1])  # 160*160*128

    # block 3
    e3 = k.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 80*80*128
    e3 = conv_block(e3, filters[2])  # 80*80*256

    # block 4
    e4 = k.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 40*40*256
    e4 = conv_block(e4, filters[3])  # 40*40*512

    # block 5
    # bottleneck layer
    e5 = k.layers.MaxPool2D(pool_size=(2, 2))(e4)  # 20*20*512
    e5 = conv_block(e5, filters[4])  # 20*20*1024

    """ Decoder """
    cat_channels = filters[0]
    cat_blocks = len(filters)
    upsample_channels = cat_blocks * cat_channels

    """ d4 """
    e1_d4 = k.layers.MaxPool2D(pool_size=(8, 8))(e1)  # 320*320*64  --> 40*40*64
    e1_d4 = conv_block(e1_d4, cat_channels, n=1)  # 320*320*64  --> 40*40*64

    e2_d4 = k.layers.MaxPool2D(pool_size=(4, 4))(e2)  # 160*160*128 --> 40*40*128
    e2_d4 = conv_block(e2_d4, cat_channels, n=1)  # 160*160*128 --> 40*40*64

    e3_d4 = k.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 80*80*256  --> 40*40*256
    e3_d4 = conv_block(e3_d4, cat_channels, n=1)  # 80*80*256  --> 40*40*64

    e4_d4 = conv_block(e4, cat_channels, n=1)  # 40*40*512  --> 40*40*64

    e5_d4 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(e5)  # 80*80*256  --> 40*40*256
    e5_d4 = conv_block(e5_d4, cat_channels, n=1)  # 20*20*1024  --> 20*20*64

    d4 = k.layers.concatenate([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = conv_block(d4, upsample_channels, n=1)  # 40*40*320  --> 40*40*320

    """ d3 """
    e1_d3 = k.layers.MaxPool2D(pool_size=(4, 4))(e1)  # 320*320*64 --> 80*80*64
    e1_d3 = conv_block(e1_d3, cat_channels, n=1)  # 80*80*64 --> 80*80*64

    e2_d3 = k.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 160*160*256 --> 80*80*256
    e2_d3 = conv_block(e2_d3, cat_channels, n=1)  # 80*80*256 --> 80*80*64

    e3_d3 = conv_block(e3, cat_channels, n=1)  # 80*80*512 --> 80*80*64

    e4_d3 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d4)  # 40*40*320 --> 80*80*320
    e4_d3 = conv_block(e4_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    e5_d3 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(e5)  # 20*20*320 --> 80*80*320
    e5_d3 = conv_block(e5_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    d3 = k.layers.concatenate([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = conv_block(d3, upsample_channels, n=1)  # 80*80*320 --> 80*80*320

    """ d2 """
    e1_d2 = k.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 320*320*64 --> 160*160*64
    e1_d2 = conv_block(e1_d2, cat_channels, n=1)  # 160*160*64 --> 160*160*64

    e2_d2 = conv_block(e2, cat_channels, n=1)  # 160*160*256 --> 160*160*64

    d3_d2 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d3)  # 80*80*320 --> 160*160*320
    d3_d2 = conv_block(d3_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d4_d2 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d4)  # 40*40*320 --> 160*160*320
    d4_d2 = conv_block(d4_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    e5_d2 = k.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(e5)  # 20*20*320 --> 160*160*320
    e5_d2 = conv_block(e5_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d2 = k.layers.concatenate([e1_d2, e2_d2, d3_d2, d4_d2, e5_d2])
    d2 = conv_block(d2, upsample_channels, n=1)  # 160*160*320 --> 160*160*320

    """ d1 """
    e1_d1 = conv_block(e1, cat_channels, n=1)  # 320*320*64 --> 320*320*64

    d2_d1 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d2)  # 160*160*320 --> 320*320*320
    d2_d1 = conv_block(d2_d1, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d3_d1 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d3)  # 80*80*320 --> 320*320*320
    d3_d1 = conv_block(d3_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d4_d1 = k.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(d4)  # 40*40*320 --> 320*320*320
    d4_d1 = conv_block(d4_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    e5_d1 = k.layers.UpSampling2D(size=(16, 16), interpolation='bilinear')(e5)  # 20*20*320 --> 320*320*320
    e5_d1 = conv_block(e5_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d1 = k.layers.concatenate([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1, ])
    d1 = conv_block(d1, upsample_channels, n=1)  # 320*320*320 --> 320*320*320

    # last layer does not have batch norm and relu
    d1 = conv_block(d1, output_channels, n=1, is_bn=False, is_relu=False)
    d1 = k.activations.softmax(d1)

    """ Deep Supervision Part"""
    if training:
        d2 = conv_block(d2, output_channels, n=1, is_bn=False, is_relu=False)
        d3 = conv_block(d3, output_channels, n=1, is_bn=False, is_relu=False)
        d4 = conv_block(d4, output_channels, n=1, is_bn=False, is_relu=False)
        e5 = conv_block(e5, output_channels, n=1, is_bn=False, is_relu=False)

        # d1 = no need for up sampling
        d2 = k.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d2)#80*80*1
        d3 = k.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d3)#20*20*1
        d4 = k.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(d4)#5*5*1
        e5 = k.layers.UpSampling2D(size=(16, 16), interpolation='bilinear')(e5)#1*1*1

        d2 = k.activations.softmax(d2)
        d3 = k.activations.softmax(d3)
        d4 = k.activations.softmax(d4)
        e5 = k.activations.softmax(e5)

    if training:
        return tf.keras.Model(inputs=input_layer, outputs=[d1, d2, d3, d4, e5], name='UNet3Plus_DeepSup')
    else:
        return tf.keras.Model(inputs=input_layer, outputs=[d1, ], name='UNet3Plus_DeepSup')


if __name__ == "__main__":
    """## Model Compilation"""
    INPUT_SHAPE = [320, 320, 1]
    OUTPUT_CHANNELS = 1

    unet_3P = unet3plus_deepsup(INPUT_SHAPE, OUTPUT_CHANNELS)
    unet_3P.summary()

    # tf.keras.utils.plot_model(unet_3P, show_layer_names=True, show_shapes=True)

    # unet_3P.save("unet_3P.hdf5")

Model: "UNet3Plus_DeepSup"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 320, 320, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 320, 320, 64)         640       ['input_layer[0][0]']         
                                                                                                  
 batch_normalization (Batch  (None, 320, 320, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 tf.nn.relu (TFOpLambda)     (None, 320, 320, 64)         0         ['batch_normal

In [ ]:

    #卷积块
  def conv_block(self,net,filters_root,n=2):
    for i in range(1, n+1):
        net = tf.keras.layers.Conv2D(
        filters=filters_root, 
        kernel_size=self.kernel_size,
        padding='same', strides=(1,1),
        kernel_regularizer=self.regularizer ,
        kernel_initializer=self.initializer)(net)

        net = tf.keras.layers.BatchNormalization()(net)
        net = tf.keras.activations.relu(net)

        net= tf.keras.layers.Dropout(rate=self.drop_rate)(net)

    return net

  def add_prediction_op(self):
    logging.info("Model: depths {depths}, filters {filters}, "
           "filter size {kernel_size[0]}x{kernel_size[1]}, "
           "pool size: {pool_size[0]}x{pool_size[1]}, "
           "dilation rate: {dilation_rate[0]}x{dilation_rate[1]}".format(
            depths=self.depths,
            filters=self.filters_root,
            kernel_size=self.kernel_size,
            dilation_rate=self.dilation_rate,
            pool_size=self.pool_size))

      # 如果 weight_decay 大于 0，设置 L2 正则化以防止过拟合，否则不使用正则化
    if self.weight_decay > 0:
      weight_decay = tf.constant(self.weight_decay, dtype=tf.float32, name="weight_constant")
      self.regularizer = tf.keras.regularizers.l2(l=0.5 * (weight_decay))
    else:
      self.regularizer = None

    #权重初始化器，1.0不缩放
    self.initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")


    filters_root = [64, 128, 256, 512, 1024]

    # net=tf.keras.layers.Input(shape=self.X_shape)# 3000,1,3
        # 使用占位符 `self.X` 作为输入张量
    with tf.compat.v1.variable_scope("Input"):
        net = self.X  # 使用占位符 `self.X` 作为网络的输入

    """ Encoder"""

    # block 1
    e1 = self.conv_block(net, filters_root[0])  # 3000*1*64

    # block 2
    e2 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e1)  # 1500*1*64
    e2 = self.conv_block(e2, filters_root[1])  # 1500*1*128

    # block 3
    e3 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e2)  # 750*1*128
    e3 = self.conv_block(e3, filters_root[2])  # 750*1*256

    # block 4
    e4 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e3)  # 375*1*256
    e4 = self.conv_block(e4, filters_root[3])  # 375*1*512

    # block 5
    # bottleneck layer
    e5 =tf.keras.layers.MaxPool2D(pool_size=(3, 1),padding='same')(e4)  # 125*1*512
    e5 = self.conv_block(e5, filters_root[4])  # 125*1*1024   

    """ Decoder """
    cat_channels = filters_root[0]
    cat_blocks = len(filters_root)
    upsample_channels = cat_blocks * cat_channels

    """ d4 """
    e1_d4 = tf.keras.layers.MaxPool2D(pool_size=(8, 1),padding='same')(e1)  # e1--3000*1*64 --> 375*1*64
    e1_d4 = self.conv_block(e1_d4,cat_channels,n=1)  # 375*1*64 --> 375*1*64

    e2_d4 = tf.keras.layers.MaxPool2D(pool_size=(4, 1),padding='same')(e2)  # 1500*1*128 --> 375*40*128
    e2_d4 = self.conv_block(e2_d4, cat_channels, n=1)  # 160*160*128 --> 40*40*64

    e3_d4 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e3)  # 750*1*256  --> 375*1*256
    e3_d4 = self.conv_block(e3_d4, cat_channels, n=1)  # 375*1*256 --> 375*1*64


    e4_d4 = self.conv_block(e4, cat_channels, n=1)  # 375*1*512  --> 375*1*64

    e5_d4 = tf.keras.layers.UpSampling2D(size=(3, 1), interpolation='bilinear')(e5)  # e5--125*1*1024  --> 375*1*1024
    e5_d4 = self.conv_block(e5_d4, cat_channels, n=1)  #375*1*1024  --> 375*1*64-----cat_channels=64

    d4 = tf.keras.layers.concatenate([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = self.conv_block(d4, upsample_channels, n=1)  # 375*1*** --> 375*1*320

    """ d3 """
    e1_d3 = tf.keras.layers.MaxPool2D(pool_size=(4, 1),padding='same')(e1)  # e1--3000*1*64 --> 750*1*64
    e1_d3 = self.conv_block(e1_d3, cat_channels, n=1)  # 750*1*64 --> 750*1*64

    e2_d3 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e2)  #  1500*1*128--> 750*1*128
    e2_d3 = self.conv_block(e2_d3, cat_channels, n=1)  # 750*1*128--> 750*1*64

    e3_d3 = self.conv_block(e3, cat_channels, n=1)  # 750*1*256 --> 750*1*64

    e4_d3 = tf.keras.layers.UpSampling2D(size=(2, 1), interpolation='bilinear')(d4)  # 375*1*320 --> 750*1*320
    e4_d3 = self.conv_block(e4_d3, cat_channels, n=1)  # 750*1*320 --> 750*1*64

    e5_d3 = tf.keras.layers.UpSampling2D(size=(6, 1), interpolation='bilinear')(e5)  # 125*1*1024  --> 750*1*1024
    e5_d3 = self.conv_block(e5_d3, cat_channels, n=1)  # 750*1*1024 --> 750*1*64

    d3 = tf.keras.layers.concatenate([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = self.conv_block(d3, upsample_channels, n=1)  # 750*1*** --> 750*1*320

    """ d2 """
    e1_d2 = tf.keras.layers.MaxPool2D(pool_size=(2, 1),padding='same')(e1)  # 3000*1*64 --> 1500*1*64
    e1_d2 = self.conv_block(e1_d2, cat_channels, n=1)  # 1500*1*64 --> 1500*1*64

    e2_d2 = self.conv_block(e2, cat_channels, n=1)  # 1500*1*128 --> 1500*1*64

    d3_d2 = tf.keras.layers.UpSampling2D(size=(2, 1), interpolation='bilinear')(d3)  # 750*1*320 --> 1500*1*320
    d3_d2 = self.conv_block(d3_d2, cat_channels, n=1)  # 1500*1*320--> 1500*1*64

    d4_d2 = tf.keras.layers.UpSampling2D(size=(4, 1), interpolation='bilinear')(d4)  # 375*1*320 --> 1500*1*320
    d4_d2 = self.conv_block(d4_d2, cat_channels, n=1)  # 1500*1*320 --> 1500*1*64

    e5_d2 = tf.keras.layers.UpSampling2D(size=(12, 1), interpolation='bilinear')(e5)  # 125*1*1024 --> 1500*1*1024
    e5_d2 = self.conv_block(e5_d2, cat_channels, n=1)  # 1500*1*1024 --> 1500*1*64

    d2 = tf.keras.layers.concatenate([e1_d2, e2_d2, d3_d2, d4_d2, e5_d2])
    d2 = self.conv_block(d2, upsample_channels, n=1)  # 1500*1*** --> 1500*1*320

    """ d1 """
    e1_d1 = self.conv_block(e1, cat_channels, n=1)  # 3000*1*64 --> 3000*1*64

    d2_d1 = tf.keras.layers.UpSampling2D(size=(2, 1), interpolation='bilinear')(d2)  # 1500*1*320 --> 3000*1*320
    d2_d1 = self.conv_block(d2_d1, cat_channels, n=1)  #  3000*1*320 -->  3000*1*64

    d3_d1 = tf.keras.layers.UpSampling2D(size=(4, 1), interpolation='bilinear')(d3)  # 750*1*320 --> 3000*1*320
    d3_d1 = self.conv_block(d3_d1, cat_channels, n=1)  # 3000*1*320 --> 3000*1*64

    d4_d1 = tf.keras.layers.UpSampling2D(size=(8, 1), interpolation='bilinear')(d4)  # 375*1*320 --> 3000*1*320
    d4_d1 = self.conv_block(d4_d1, cat_channels, n=1)  # 3000*1*320--> 3000*1*64

    e5_d1 = tf.keras.layers.UpSampling2D(size=(24, 1), interpolation='bilinear')(e5)  # 125*1*1024--> 3000*1*1024
    e5_d1 = self.conv_block(e5_d1, cat_channels, n=1)  # 3000*1*1024 --> 3000*1*64

    d1 = tf.keras.layers.concatenate([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1, ])
    d1 = self.conv_block(d1, upsample_channels, n=1)  #3000*1*** --> 3000*1*320

    # # 最后一层没有归一化和ReLU

    d=tf.keras.layers.Conv2D(self.n_class, kernel_size=(3, 1), padding='same', activation=None)(d1)


    output = tf.keras.activations.softsign(d)
      
    # last layer does not have batch norm and relu
    # d1 = self.conv_block(d1, self.n_class, n=1)
    # d1 = tf.keras.activations.softmax(d1)

    # """ Deep Supervision Part"""

    # d2=tf.keras.layers.Conv2D(self.n_class,kernel_size=self.kernel_size, padding='same', activation=None)(d2)
    # d3=tf.keras.layers.Conv2D(self.n_class,kernel_size=self.kernel_size, padding='same', activation=None)(d3)
    # d4=tf.keras.layers.Conv2D(self.n_class,kernel_size=self.kernel_size, padding='same', activation=None)(d4)
    # e5=tf.keras.layers.Conv2D(self.n_class,kernel_size=self.kernel_size, padding='same', activation=None)(e5)
    

    # # d1 = no need for up sampling
    # d2 = tf.keras.layers.UpSampling2D(size=(2, 1), interpolation='bilinear')(d2)
    # d3 = tf.keras.layers.UpSampling2D(size=(4, 1), interpolation='bilinear')(d3)
    # d4 = tf.keras.layers.UpSampling2D(size=(8, 1), interpolation='bilinear')(d4)
    # e5 = tf.keras.layers.UpSampling2D(size=(24, 1), interpolation='bilinear')(e5)

    # d2 = tf.keras.activations.softmax(d2)
    # d3 = tf.keras.activations.softmax(d3)
    # d4 = tf.keras.activations.softmax(d4)
    # e5 = tf.keras.activations.softmax(e5)


    
    

      # 保存编码器部分的最终表示，可以用于其他任务或分析
    # with tf.compat.v1.variable_scope("representation"):
    #   self.representation = convs[-1]

    #logits 是最终的网络输出，未经过激活函数
    with tf.compat.v1.variable_scope("logits"):
      self.logits = output
      # 记录 logits 的直方图摘要，以便在 TensorBoard 中可视化
      tmp = tf.compat.v1.summary.histogram("logits", self.logits)
      self.summary_train.append(tmp)


    # 计算最终预测结果，使用 softmax 将 logits 转换为概率分布
    with tf.compat.v1.variable_scope("preds"):
      self.preds = tf.nn.softmax(output)
      # 记录预测结果的直方图摘要，以便在 TensorBoard 中可视化
      tmp = tf.compat.v1.summary.histogram("preds", self.preds)
      self.summary_train.append(tmp)